In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import io

# Paste your CSV data here
csv_data = """
Experiment_ID;Base_Model_Arch;Optimization_Category;Specific_Technique;Key_Parameters;Top_1_Acc;Top_5_Acc;Model_Size_MB_Disk;FT_Epochs_Run;FT_Time_seconds;Inference_Time_ms_CPU (Batch 1);Inference_Time_ms_GPU (Batch 1);flops_gmacs;params_millions;non_zero_params;Notes_from_Log
resnet18_baseline;ResNet18;Baseline;Baseline;N/A;0.5008921743563599;0.7631914;44.66958;15;1406.56149;35.31372;3.52352;1.824033792;11.689512;11.689512;
resnet50_baseline;ResNet50;Baseline;Baseline;N/A;0.649502931430028;0.8748407;97.79614;30;5395.38120;88.06000;7.87376;4.133742592;25.557032;25.557032;
resnet18pretrained_distilled_quant_kmeans_256clusters_post;ResNet18;Combined;KMeans Quant;Clusters: 256;0.5368340555697171;0.8032118;44.66733;0;0.00000;37.22856;3.33568;1.824033792;11.689512;11.689512;K-Means Weight Quantization (256 clusters) applied to a distilled, pretrained ResNet18. No training epochs for quantization step. Evaluation metrics from original log. Codebook info was generated.
resnet18pretrained_distilled_quant_ptq_int8_perchannel_post;ResNet18;Combined;PTQ INT8 (Per-Channel);N/A;0.5077746622482794;0.7807800;11.30209;0;0.00000;19.01976;N/A (Known JIT GPU Unstable);N/A (JIT Model);N/A (JIT Model);N/A (JIT Model);Post-Training Quantization (per-channel) applied to a distilled, pretrained ResNet18. No training epochs for quantization step. Evaluation metrics from original log.
resnet18pretrained_distilled_quant_ptq_int8_pertensor_post;ResNet18;Combined;PTQ INT8 (Per-Tensor);N/A;0.5335202651032374;0.8027020;11.30076;0;0.00000;18.31738;N/A (Known JIT GPU Unstable);N/A (JIT Model);N/A (JIT Model);N/A (JIT Model);Post-Training Quantization (per-tensor) applied to a distilled, pretrained ResNet18. No training epochs for quantization step. Evaluation metrics from original log.
resnet18pretrained_distilled_quant_qat_int8_epochs8;ResNet18;Combined;QAT INT8;QAT Epochs: 8;0.5355595207749172;0.8044864;11.29939;8;1155.46389;17.96892;N/A (Known JIT GPU Unstable);N/A (JIT Model);N/A (JIT Model);N/A (JIT Model);Quantization Aware Training applied to a distilled, pretrained ResNet18. Epoch data refers to QAT fine-tuning. Evaluation metrics from original log.
resnet50_to_resnet18pretrained_kd;ResNet18;Knowledge Distillation;Knowledge Distillation;T:resnet50->S:resnet18;0.538108590364517;0.8027020;44.67078;15;2067.91055;37.01086;3.47354;1.824033792;11.689512;11.689512;
resnet50_to_resnet18scratch_kd;ResNet18;Knowledge Distillation;Knowledge Distillation;T:resnet50->S:resnet18;0.1763956156003059;0.4017334;44.67078;15;2004.46795;36.02530;3.41316;1.824033792;11.689512;11.689512;
resnet50_prune_nm24_ft;ResNet50;Pruning;N:M Sparsity;N:2, M:4;0.6426204435381085;0.8730563;97.79861;15;3051.65437;89.32968;9.58856;4.133742592;25.557032;13.829576;
resnet50_prune_struct_it_l1filter_stage1_approx_sp50_ft;ResNet50;Pruning;Iterative Structured Pruning (L1 Filter);Target Sparsity: 50.0%;0.49936273260260006;0.7675249;49.60373;5;675.39339;57.95676;8.98030;2.042114256;12.935549;12.935549;
resnet50_prune_struct_it_l1filter_stage2_approx_sp75_ft;ResNet50;Pruning;Iterative Structured Pruning (L1 Filter);Target Sparsity: 75.0%;0.29824114198317614;0.5799133;25.49252;10;1021.56831;37.59312;11.46844;1.012597635;6.626072;6.626072;
resnet50_prune_struct_it_l1filter_stage3_approx_sp90_ft;ResNet50;Pruning;Iterative Structured Pruning (L1 Filter);Target Sparsity: 90.0%;0.13076518225850832;0.3405557;10.21170;15;1358.46598;24.75034;8.29152;0.382400928;2.630579;2.630579;
resnet50_prune_struct_os_l1filter_fp30_ft;ResNet50;Pruning;One-Shot Structured Pruning (L1 Filter);Target Sparsity: 50.0%;0.5477950548049961;0.8100943;49.59605;15;2030.61859;56.12866;9.36346;2.042114256;12.935549;12.935549;
resnet50_prune_struct_os_l1filter_fp55_ft;ResNet50;Pruning;One-Shot Structured Pruning (L1 Filter);Target Sparsity: 75.0%;0.22074936273260006;0.4840683;21.86064;15;1472.21088;35.38324;8.50816;0.865708368;5.678189;5.678189;
resnet50_prune_struct_os_l1filter_fp70_ft;ResNet50;Pruning;One-Shot Structured Pruning (L1 Filter);Target Sparsity: 90.0%;0.05709915880703543;0.1947489;10.56108;15;1346.40862;25.03054;8.32954;0.400809619;2.724022;2.724022;
resnet50_prune_unstruct_it_l1_stage1_sp50_ft;ResNet50;Pruning;Iterative Unstructured Pruning (L1);Target Sparsity: 50.0%;0.6605647208768799389;0.853138;97.79218;5;665.13087;88.04078;8.22656;4.133742592;25.557032;12.752456;
resnet50_prune_unstruct_it_l1_stage2_sp75_ft;ResNet50;Pruning;Iterative Unstructured Pruning (L1);Target Sparsity: 75.0%;0.6194745347947998981;0.79276;97.79218;10;1337.13718;86.65992;7.89072;4.133742592;25.557032;6.376728;
resnet50_prune_unstruct_it_l1_stage3_sp90_ft;ResNet50;Pruning;Iterative Unstructured Pruning (L1);Target Sparsity: 90.0%;0.4649502931430028;0.7519755;97.80362;15;2093.07571;94.62280;7.99122;4.133742592;25.557032;2.551291;
resnet50_prune_unstruct_os_l1_sp50_ft;ResNet50;Pruning;One-Shot Unstructured Pruning (L1);Target Sparsity: 50.0%;0.6645424420086669;0.8860566;97.80331;15;2085.63127;90.80236;8.09102;4.133742592;25.557032;12.752456;
resnet50_prune_unstruct_os_l1_sp75_ft;ResNet50;Pruning;One-Shot Unstructured Pruning (L1);Target Sparsity: 75.0%;0.6240122355340301;0.8598012;97.80331;15;1945.38252;103.82330;8.23356;4.133742592;25.557032;6.376728;
resnet50_prune_unstruct_os_l1_sp90_ft;ResNet50;Pruning;One-Shot Unstructured Pruning (L1);Target Sparsity: 90.0%;0.41830231965332654;0.7122100;97.80331;15;2004.42808;89.49998;8.24150;4.133742592;25.557032;2.551291;
resnet50_quant_kmeans_256clusters_post;ResNet50;Quantization;KMeans Quant;Clusters: 256;0.6492480244710681;0.8730563;97.79564;0;0.00000;89.61634;9.13594;4.133742592;25.557032;25.557032;K-Means Weight Quantization (256 clusters). No training epochs for quantization step. Evaluation metrics from original log. Codebook info was generated.
resnet50_quant_ptq_int8_perchannel_post;ResNet50;Quantization;PTQ INT8 (Per-Channel);N/A;0.6069334692837114;0.8404282;24.94237;0;0.00000;44.42032;N/A (Known JIT GPU Unstable);N/A (JIT Model);N/A (JIT Model);N/A (JIT Model);Post-Training Quantization (per-channel). No training epochs for quantization step. Evaluation metrics from original log.
resnet50_quant_ptq_int8_pertensor_post;ResNet50;Quantization;PTQ INT8 (Per-Tensor);N/A;0.649757838388988;0.8735661;24.94018;0;0.00000;41.93892;N/A (Known JIT GPU Unstable);N/A (JIT Model);N/A (JIT Model);N/A (JIT Model);Post-Training Quantization (per-tensor). No training epochs for quantization step. Evaluation metrics from original log.
resnet50_quant_qat_int8_epochs8;ResNet50;Quantization;QAT INT8;QAT Epochs: 8;0.6635228141728269;0.8807035;24.93738;8;2333.11918;42.46538;N/A (Known JIT GPU Unstable);N/A (JIT Model);N/A (JIT Model);N/A (JIT Model);Quantization Aware Training. Epoch data refers to QAT fine-tuning. Evaluation metrics from original log.
resnet50_trt_fp16;ResNet50;TensorRT;TensorRT FP16;Precision: FP16;0.6505;0.8746;67.38;0;0.0;;6.36;4.133742592;25.557032;25.557032;TensorRT FP16 optimization applied to ResNet50 baseline.
resnet50_trt_fp32;ResNet50;TensorRT;TensorRT FP32;Precision: FP32;0.6492;0.8751;162.20;0;0.0;;18.15;4.133742592;25.557032;25.557032;TensorRT FP32 optimization applied to ResNet50 baseline.
resnet18_distilled_trt_fp16;ResNet18;Combined;Distillation + TensorRT FP16;Precision: FP16;0.53836;0.80270;31.02;0;0.0;;2.52;1.824033792;11.689512;11.689512;TensorRT FP16 optimization applied to a distilled, pretrained ResNet18.
resnet18_distilled_trt_fp32;ResNet18;Combined;Distillation + TensorRT FP32;Precision: FP32;0.53785;0.80270;96.01;0;0.0;;8.12;1.824033792;11.689512;11.689512;TensorRT FP32 optimization applied to a distilled, pretrained ResNet18.
;;;;;;;;;;;;;;;
"""

# --- 1. Data Loading and Cleaning ---
df = pd.read_csv(io.StringIO(csv_data), sep=';')
df.dropna(how='all', inplace=True)
df.columns = df.columns.str.replace(' \(.*\)', '', regex=True).str.replace(' ', '_')

numeric_cols = [
    'Top_1_Acc', 'Top_5_Acc', 'Model_Size_MB_Disk', 'FT_Epochs_Run',
    'FT_Time_seconds', 'Inference_Time_ms_CPU', 'Inference_Time_ms_GPU',
    'flops_gmacs', 'params_millions', 'non_zero_params'
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['Top_1_Acc_Percent'] = df['Top_1_Acc'] * 100
df['Top_5_Acc_Percent'] = df['Top_5_Acc'] * 100

# ----------------- FIX: Handle NaN values for size parameters -----------------
# The 'size' parameter in Plotly cannot handle NaN values. We will fill them
# with 0, so these points are still plotted but with a minimum size.
# This affects both 'params_millions' for the first plot and 'flops_gmacs' for the second.
df['params_millions'].fillna(0, inplace=True)
df['flops_gmacs'].fillna(0, inplace=True)
# -------------------------------------------------------------------------------


# --- 2. Create the Interactive Plot ---

# Plot 1: Accuracy vs. Model Size
fig1 = px.scatter(
    df,
    x="Model_Size_MB_Disk",
    y="Top_1_Acc_Percent",
    color="Optimization_Category",
    symbol="Base_Model_Arch",
    size="params_millions",
    hover_name="Experiment_ID",
    hover_data={
        "Specific_Technique": True,
        "Top_1_Acc_Percent": ':.2f}%',
        "Model_Size_MB_Disk": ':.2f} MB',
        "Inference_Time_ms_CPU": ':.2f} ms',
        "params_millions": ':.2f}M'
    },
    template="plotly_dark",
    title="<b>Model Performance vs. Size</b> (Accuracy vs. Disk Footprint)"
)

fig1.update_layout(
    xaxis_title="Model Size on Disk (MB)",
    yaxis_title="Top-1 Accuracy (%)",
    legend_title="Optimization Category",
    font=dict(family="Arial, sans-serif", size=12)
)
fig1.update_traces(marker=dict(line=dict(width=1, color='White')))

# Plot 2: Accuracy vs. CPU Inference Time
fig2 = px.scatter(
    df,
    x="Inference_Time_ms_CPU",
    y="Top_1_Acc_Percent",
    color="Optimization_Category",
    symbol="Base_Model_Arch",
    size="flops_gmacs",
    hover_name="Experiment_ID",
    hover_data={
        "Specific_Technique": True,
        "Top_1_Acc_Percent": ':.2f}%',
        "Model_Size_MB_Disk": ':.2f} MB',
        "Inference_Time_ms_CPU": ':.2f} ms',
        "flops_gmacs": ':.2f}G'
    },
    template="plotly_dark",
    title="<b>Model Performance vs. Speed</b> (Accuracy vs. CPU Inference Time)"
)

fig2.update_layout(
    xaxis_title="CPU Inference Time (ms, Batch Size 1)",
    yaxis_title="Top-1 Accuracy (%)",
    legend_title="Optimization Category",
    font=dict(family="Arial, sans-serif", size=12)
)
fig2.update_traces(marker=dict(line=dict(width=1, color='White')))


# --- 3. Save and Show the Plots ---
fig1.write_html("model_performance_vs_size.html")
fig2.write_html("model_performance_vs_speed.html")

fig1.show()
fig2.show()

print("Interactive plots have been saved as 'model_performance_vs_size.html' and 'model_performance_vs_speed.html'")

Interactive plots have been saved as 'model_performance_vs_size.html' and 'model_performance_vs_speed.html'


In [2]:
# --- Idea 1: Sunburst Chart ---
# (Add this code to your existing script after the data cleaning)

sunburst_fig = px.sunburst(
    df.dropna(subset=['Top_1_Acc_Percent', 'Model_Size_MB_Disk']), # Drop rows where color/value is NaN
    path=['Base_Model_Arch', 'Optimization_Category', 'Specific_Technique'],
    values='Model_Size_MB_Disk',
    color='Top_1_Acc_Percent',
    color_continuous_scale='RdYlGn', # Red-Yellow-Green scale shows bad-to-good
    hover_name='Specific_Technique',
    hover_data={'Model_Size_MB_Disk': ':.2f} MB'},
    title='<b>Hierarchical Overview of Optimization Experiments</b><br>(Slice size = Model Size, Color = Top-1 Accuracy)',
    template='plotly_dark'
)

sunburst_fig.show()
sunburst_fig.write_html("overview_sunburst.html")

In [3]:
# --- Idea 2: Box Plot Analysis ---
# (This requires a bit of data manipulation first)

# Find the baseline metrics for each architecture
baselines = df[df['Optimization_Category'] == 'Baseline'].set_index('Base_Model_Arch')
baseline_metrics = baselines[['Top_1_Acc', 'Model_Size_MB_Disk']].to_dict('index')
# baseline_metrics will look like:
# {'ResNet18': {'Top_1_Acc': 0.50, 'Model_Size_MB_Disk': 44.6}, ...}

# Map the baseline values to each row in the original dataframe
df['baseline_acc'] = df['Base_Model_Arch'].map(lambda x: baseline_metrics[x]['Top_1_Acc'])
df['baseline_size'] = df['Base_Model_Arch'].map(lambda x: baseline_metrics[x]['Model_Size_MB_Disk'])

# Calculate the percentage change relative to the baseline
# We exclude the baseline itself from this plot
plot_df = df[df['Optimization_Category'] != 'Baseline'].copy()
plot_df['Accuracy_Change_%'] = ((plot_df['Top_1_Acc'] - plot_df['baseline_acc']) / plot_df['baseline_acc']) * 100
plot_df['Size_Reduction_%'] = ((plot_df['baseline_size'] - plot_df['Model_Size_MB_Disk']) / plot_df['baseline_size']) * 100

# Create the box plot
box_fig = px.box(
    plot_df,
    x="Optimization_Category",
    y="Size_Reduction_%",
    color="Optimization_Category",
    points="all", # Show all the individual experiment points
    facet_col="Base_Model_Arch", # Create separate plots for ResNet18 and ResNet50
    hover_name="Experiment_ID",
    hover_data=["Accuracy_Change_%"],
    title="<b>Effectiveness of Optimization Categories</b> (Relative to Baseline)",
    template='plotly_dark'
)

box_fig.update_yaxes(title_text="Model Size Reduction (%)")
box_fig.update_xaxes(title_text="")
box_fig.show()
box_fig.write_html("effectiveness_boxplot.html")

In [4]:
# --- Idea 3: Pruning Degradation Line Chart ---
# (Requires extracting the sparsity value from 'Key_Parameters')

pruning_df = df[df['Optimization_Category'] == 'Pruning'].copy()

# Extract sparsity percentage using regex. This is a powerful pandas trick.
pruning_df['Sparsity_%'] = pruning_df['Key_Parameters'].str.extract(r'(\d+\.?\d*)%').astype(float)

# Create a more descriptive pruning method name
pruning_df['Method'] = pruning_df['Specific_Technique'].apply(
    lambda x: ' '.join(x.split(' ')[:3]) # e.g., "Iterative Structured Pruning"
)

# Create the line plot
line_fig = px.line(
    pruning_df.sort_values('Sparsity_%'),
    x="Sparsity_%",
    y="Top_1_Acc_Percent",
    color="Method",
    symbol="Method",
    markers=True, # Add markers to the points
    title="<b>Accuracy vs. Pruning Sparsity on ResNet50</b>",
    template='plotly_dark'
)
line_fig.update_xaxes(title_text="Target Sparsity (%)")
line_fig.update_yaxes(title_text="Top-1 Accuracy (%)")
line_fig.show()
line_fig.write_html("pruning_degradation.html")

In [5]:
# --- Idea 4: Speed vs. Size Bubble Chart ---
# (Add this code to your existing script after the data cleaning and NaN filling)

# We need to drop rows where inference time is NaN to plot them.
plot_df_speed = df.dropna(subset=['Inference_Time_ms_CPU', 'Model_Size_MB_Disk', 'Top_1_Acc_Percent']).copy()

speed_size_fig = px.scatter(
    plot_df_speed,
    x="Model_Size_MB_Disk",
    y="Inference_Time_ms_CPU",
    size="Top_1_Acc_Percent",
    color="Optimization_Category",
    symbol="Base_Model_Arch",
    hover_name="Experiment_ID",
    hover_data={
        "Top_1_Acc_Percent": ':.2f}%',
        "Model_Size_MB_Disk": ':.2f} MB',
        "Inference_Time_ms_CPU": ':.2f} ms'
    },
    size_max=60, # Increase max bubble size for better visual impact
    title="<b>Deployment Profile: Inference Speed vs. Model Size</b><br>(Bubble size represents Accuracy)",
    template="plotly_dark"
)

speed_size_fig.update_layout(
    xaxis_title="Model Size on Disk (MB)",
    yaxis_title="CPU Inference Time (ms)",
    legend_title="Optimization Category"
)
speed_size_fig.show()
speed_size_fig.write_html("speed_vs_size_profile.html")

In [11]:
# --- Idea 5: Parallel Categories Diagram (Final Robust Version) ---

# Create a copy for this plot
para_cat_df = df.dropna(subset=[
    'flops_gmacs', 'Model_Size_MB_Disk', 'Inference_Time_ms_CPU'
]).copy()

# Bin the numerical data and immediately convert the resulting Interval objects to strings
para_cat_df['FLOPs_Category'] = pd.qcut(
    para_cat_df['flops_gmacs'],
    q=4,
    duplicates='drop'
).astype(str) # <-- FIX: Convert to string

para_cat_df['Size_Category'] = pd.qcut(
    para_cat_df['Model_Size_MB_Disk'],
    q=4,
    duplicates='drop'
).astype(str) # <-- FIX: Convert to string

para_cat_df['Speed_Category'] = pd.qcut(
    para_cat_df['Inference_Time_ms_CPU'],
    q=4,
    duplicates='drop'
).astype(str) # <-- FIX: Convert to string

# Now, create the plot
parallel_cat_fig = px.parallel_categories(
    para_cat_df,
    dimensions=['Optimization_Category', 'Base_Model_Arch', 'FLOPs_Category', 'Size_Category', 'Speed_Category'],
    color="Top_1_Acc_Percent",
    color_continuous_scale=px.colors.sequential.YlGn,
    title="<b>Relationships Between Theoretical and Practical Costs</b>",
    template='plotly_dark'
)

# Improve label readability for interval-based names
parallel_cat_fig.update_layout(
    margin=dict(l=100, r=100, t=100, b=100)
)

parallel_cat_fig.show()
parallel_cat_fig.write_html("cost_relationships_parallel_categories.html")

In [7]:
# --- Idea 6: Training Cost Bar Chart ---

# Filter for experiments that actually had a fine-tuning step
training_cost_df = df[df['FT_Time_seconds'] > 0].copy()

# Convert seconds to minutes for better readability
training_cost_df['FT_Time_minutes'] = training_cost_df['FT_Time_seconds'] / 60

bar_fig = px.bar(
    training_cost_df.sort_values('FT_Time_minutes'),
    x="Experiment_ID",
    y="FT_Time_minutes",
    color="Optimization_Category",
    title="<b>Fine-Tuning Time Overhead for Optimization Techniques</b>",
    labels={'FT_Time_minutes': 'Fine-Tuning Time (Minutes)', 'Experiment_ID': 'Experiment'},
    template='plotly_dark',
    hover_data=['FT_Epochs_Run', 'Top_1_Acc_Percent']
)
bar_fig.update_xaxes(tickangle=45) # Angle the x-axis labels to prevent overlap
bar_fig.show()
bar_fig.write_html("training_cost_barchart.html")